In [25]:
table<-read.csv("feattable.csv")

In [26]:
#dont change things here
table$row.m.z<-round(table$row.m.z, 4) 
table$row.retention.time<-round(table$row.retention.time, 2) 
sampleid <-numeric()
for(i in 1:nrow(table)){
  id<-c(table$row.m.z[i], table$row.retention.time[i], table$row.ID[i])
  nums<-c("X", paste(id, collapse = "_"))
  sampleid[i]<-paste(nums, collapse = "")
}
tab.w.id<-cbind(sampleid, table[, -(1:3)])
names(tab.w.id) <- sub(".mzML.Peak.area|.mzXML.Peak.area", "", names(tab.w.id))
trans<-t(tab.w.id)
rownames(trans)[1] <- "sampleid"
trans<-cbind(row.names(trans), trans)
colnames(trans)<-trans[1,]
trans<-trans[-1,]
trans[is.na(trans)] <- 0     #this replaces any "NA" output with zero 
trans <- trans[rownames(trans) != "X|X.", ]
#removes .mzML Peak Area from file name, removes old feature columns, adds in mz_RT_CI format for feature, transposes

In [27]:
write.table(trans, "FeatureTable_QiimeFormat.txt", row.names = FALSE, sep = "\t", na="") #make sure its .txt. NOT .csv

In [28]:
#now.. optional checking to make sure the Permanova job doesn't fail

In [29]:
meta<-read.csv("metadata.csv") #input metadata .csv
trans<-read.table("FeatureTable_QiimeFormat.txt", sep="\t", header = TRUE) #from previous step

In [30]:
##part 1##
names(meta)[1] <- "sampleid"
metasamps<-as.vector(meta$sampleid) 
featlist_samples<-trans[ ,1]
unique_to_meta <- setdiff(metasamps, featlist_samples)
cat("Unique to Metadata file:", unique_to_meta, "\n")
unique_to_featlist <- setdiff(featlist_samples, metasamps)
cat("Unique to Featlist file:", unique_to_featlist, "\n")
#prints the filenames that dont have matches in the other file. does not work if one file has an exact duplicate
#youll have to edit the excel file if names are printed


Unique to Metadata file:  
Unique to Featlist file:  


In [31]:
##part 2##
#making sure engths are equal
length(metasamps)
length(featlist_samples)


[1] 6

[1] 6

In [32]:
##part3##
#checking for duplicates in featlist
duplicatesf <- duplicated(featlist_samples)
duplicate_valuesf <- unique(featlist_samples[duplicatesf])
if (any(duplicatesf)) {
  cat("Duplicate values found in featlist:", duplicate_valuesf, "\n")
} else {cat("No duplicate values found in featlist\n")}


No duplicate values found in featlist


In [33]:
##part 4##
#checking for duplicates in metadata
duplicatesm <- duplicated(metasamps)
duplicate_valuesm <- unique(metasamps[duplicatesm])
if (any(duplicatesm)) {
  cat("Duplicate values found in metadata:", duplicate_valuesm, "\n")
} else {cat("No duplicate values found in metadata\n")}

No duplicate values found in metadata


In [34]:
##part 5##
#only works if cells above have found no issues
#Qiime comes up with errors if one of the samples has no features present.(sum of 0)
#This code IDs the samples with a total row sum of zero, then deletes that sample from both the feature list and the metadata
rowsums<-numeric()
for(i in 1:nrow(trans)){
  sum<-sum(as.numeric(trans[i, 2:ncol(trans)]))
  rowsums[i]<-round(sum)
}
dftrans<-as.data.frame(trans)
dfsums<-as.data.frame(cbind(rownames(dftrans), rowsums))
samp_to_remov<- dfsums$V1[dfsums$rowsums == 0]
featlist_filtered<- trans[!(dftrans$sampleid %in% samp_to_remov), ]
metadata_filtered<- meta[!(meta$sampleid %in% samp_to_remov), ]

##part 6##
#this prints results of the cell. it will print the sample names that have been removed(it will likely be blanks)
#Then the length() function tells you how many.
dfsums$V1[dfsums$rowsums == 0]
length(dfsums$V1[dfsums$rowsums == 0])

character(0)

[1] 0

In [35]:
#export. keep the .txt
write.table(featlist_filtered, "correctedfeatlist.txt", row.names = FALSE, sep = "\t", na="")
write.table(metadata_filtered, "correctedmetadata.txt", row.names = FALSE, sep = "\t", na="")


In [15]:
#Another optional section- if you want to subset your data, heres a few ways to do it

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!Do not run everything..choose what you want to change about your table. then choose one of the pre-made codes to run. 
#make sure to check your files to make sure 

In [36]:
#input the metadata and feature table from the format checking output
metadata<-read.table("correctedmetadata.txt", sep="\t", header = TRUE)
featureTable<-read.table("correctedfeatlist.txt", sep="\t", header = TRUE)


In [37]:
#if you want to drop sections. #change to whatever pattern you want in the sample names. This line is case sensitive. 
to_drop <- "blank|Blank|BLANK|QC|Qc|qc|6mix" #change this line
featTable_filtered <- subset(featureTable, !grepl(to_drop, sampleid, ignore.case = TRUE))
metadata_filtered <- subset(metadata, metadata$sampleid %in% featTable_filtered$sampleid)

In [21]:
#if you only want to include 1 group from metadata
metadata_filtered <- metadata[metadata$organ == "heart", ] #change this to your column name and group you want to keep
featTable_filtered <- subset(featureTable, featureTable$sampleid %in% metadata_filtered$sampleid )

In [38]:
#print for checking you have the groups you want
metadata_filtered
featTable_filtered

,sampleid,type,group,batch
,<chr>,<chr>,<chr>,<chr>
1,SampleB3,sample,infected,A
3,SampleB4,sample,uninfected,B
4,SampleB5,sample,uninfected,B
5,SampleB2,sample,infected,A
6,SampleB1,sample,infected,A


,sampleid,X255.233_1.32_1,X283.2643_1.3_2,X475.1898_5.06_3,X477.2102_1.29_4,X218.1034_4.27_5,X497.1717_5.05_6,X307.1124_1.45_7,X783.3523_1.36_8,X277.2174_1.15_9,...,X118.9421_8.84_719,X184.8586_6.52_722,X210.8424_2.5_723,X212.0751_11.97_726,X245.0546_6.32_728,X152.8806_2.07_730,X112.9857_6.93_734,X246.0573_4.47_736,X246.0573_4.13_737,X212.0751_10.45_740
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,SampleB3,0.08467591,0.05174543,7.858401e-02,0.02124733,0.03478147,5.578356e-02,0.02383133,0.011127700,0.010856360,...,0.0000000000,0.0000000000,0.000000000,0.0000000000,0.000000e+00,0.000000e+00,0.000000000,0.000000000,0.000000000,0.000000000
2,SampleB4,0.09763448,0.07680741,1.002592e-01,0.01274640,0.00000000,4.876361e-02,0.02257547,0.006292860,0.011635870,...,0.0000000000,0.0000000000,0.000000000,0.0000000000,0.000000e+00,0.000000e+00,0.000000000,0.000000000,0.000000000,0.000000000
3,SampleB5,0.09633782,0.05373095,2.023849e-05,0.01305176,0.00000000,5.844279e-05,0.00000000,0.006540454,0.000000000,...,0.0000000000,0.0000000000,0.000000000,0.0000000000,0.000000e+00,0.000000e+00,0.000000000,0.000000000,0.000000000,0.000000000
4,SampleB2,0.06999471,0.05893756,1.173058e-01,0.01053842,0.00000000,0.000000e+00,0.01977300,0.004336383,0.005285657,...,0.0004270462,0.0010981930,0.001203776,0.0006102965,1.382095e-05,1.232620e-05,0.000000000,0.000000000,0.000000000,0.000000000
5,SampleB1,0.08281807,0.04886171,5.897701e-02,0.01721257,0.02253764,8.800301e-02,0.00000000,0.005830865,0.009354129,...,0.0003728453,0.0002251345,0.001601254,0.0009127267,6.674082e-05,2.616419e-05,0.001719821,0.003357335,0.001841044,0.001059833


In [39]:
###exporting
#for Qiime format             
write.table(featTable_filtered, "subset_featlist.txt", row.names = FALSE, sep = "\t", na="")
write.table(metadata_filtered, "subset_metadata.txt", row.names = FALSE, sep = "\t", na="")

In [40]:
#for regular .csv
write.csv(featTable_filtered, "subset_featlist.csv", row.names = FALSE) 
write.csv(metadata_filtered, "subset_metadata.csv", row.names = FALSE) 